In [ ]:
import pandas as pd
import seaborn as sns

In [ ]:
df = pd.read_csv(r'C:\Users\tjagg\Downloads\archive\TWO_CENTURIES_OF_UM_RACES.csv', encoding= 'unicode_escape')

In [ ]:
df.head(10)

In [ ]:
df.shape

In [ ]:
df.dtypes

In [ ]:
df.info()

In [ ]:
# Only want USA Races, 50Km or 50miles, 2020

In [ ]:
# 1) show 50Km or 50Mi
# 50Km
# 50mi

In [ ]:
df[df['Event distance/length'] == '50mi']

In [ ]:
# Combine 50km/50mi with isin

In [ ]:
df[df['Event distance/length'].isin(['50km','50mi'])]

In [ ]:
# 2) Then break down to 2020 year
df[(df['Event distance/length'].isin(['50km','50mi'])) & (df['Year of event'] == 2020)]

In [ ]:
# 3) USA only
df[df['Event name'] == 'Everglades 50 Mile Ultra Run (USA)']['Event name'].str.split('(').str.get(1).str.split(')').str.get(0)

In [ ]:
df[df['Event name'].str.split('(').str.get(1).str.split(')').str.get(0) == 'USA']

In [ ]:
# combine all filters together

In [ ]:
df[(df['Event distance/length'].isin(['50km','50mi'])) & (df['Year of event'] == 2020) & (df['Event name'].str.split('(').str.get(1).str.split(')').str.get(0) == 'USA')]

In [ ]:
df2 = df[(df['Event distance/length'].isin(['50km','50mi'])) & (df['Year of event'] == 2020) & (df['Event name'].str.split('(').str.get(1).str.split(')').str.get(0) == 'USA')]

In [ ]:
df2.head(10)

In [ ]:
df2.shape

In [ ]:
df2.info()

In [ ]:
# removing (USA) from the event name

In [ ]:
df2['Event name'].str.split('(').str.get(0)

In [ ]:
df2['Event name'] = df2['Event name'].str.split('(').str.get(0)

In [ ]:
df2.head(10)

In [ ]:
# clean up athlete age

In [ ]:
df2['athlete_age'] = 2020 - df2['Athlete year of birth']

In [ ]:
# remove h from athlete performance

In [ ]:
df2['Athlete performance'] = df2['Athlete performance'].str.split(' ').str.get(0)

In [ ]:
df2.head()

In [ ]:
# drop columns: Athlete Club, Athlete Country, Athlete year of birth, Athlete Age Category

In [ ]:
df2 = df2.drop(['Athlete club', 'Athlete country', 'Athlete year of birth', 'Athlete age category'], axis = 1)

In [ ]:
df2.head()

In [ ]:
# clean up null values

In [ ]:
df2.isna().sum()

In [ ]:
df2[df2['athlete_age'].isna()==1]

In [ ]:
df2 = df2.dropna()

In [ ]:
df2.shape

In [ ]:
# Check for duplicate values

In [ ]:
df2[df2.duplicated() == True]

In [ ]:
# reset index

In [ ]:
df2.reset_index(drop = True)

In [ ]:
# fix types

In [ ]:
df2.dtypes

In [ ]:
df2['athlete_age'] = df2['athlete_age'].astype(int)

In [ ]:
df2['Athlete average speed'] = df2['Athlete average speed'].astype(float)

In [ ]:
df2.dtypes

In [ ]:
df2.head()

In [ ]:
# rename columns

In [ ]:
df2 = df2.rename(columns = {'Year of event':'year',
                          'Event dates':'race_day',
                          'Event name':'race_name',
                          'Event distance/length':'race_length',
                          'Event number of finishers':'race_number_of_finishers',
                          'Athlete performance':'athlete_performance',
                          'Athlete gender':'athlete_gender',
                          'Athlete average speed':'athlete_average_speed',
                          'Athlete ID':'athlete_id'
                          })

In [ ]:
df2.head()

In [ ]:
# reorder columns

In [ ]:
df3 = df2[['race_day','race_name','race_length','race_number_of_finishers','athlete_id','athlete_gender','athlete_age','athlete_performance','athlete_average_speed','year']]

In [ ]:
df3.head()

In [ ]:
df3[df3['race_name'] == 'Everglades 50 Mile Ultra Run ']

In [ ]:
df3[df3['athlete_id'] == 222509]

In [ ]:
# Exploratory Data Analysis

In [ ]:
# charts and graphs

In [ ]:
sns.histplot(df3['race_length'])

In [ ]:
sns.histplot(df3, x = 'race_length', hue = 'athlete_gender')

In [ ]:
sns.displot(df3[df3['race_length'] == '50mi']['athlete_average_speed'])

In [ ]:
sns.violinplot(data = df3, x = 'race_length', y = 'athlete_average_speed', hue = 'athlete_gender', split=True, inner = 'quart', linewidth = 1)

In [ ]:
sns.lmplot(data = df3, x = 'athlete_age', y = 'athlete_average_speed', hue = 'athlete_gender')

In [ ]:
# 1) Difference in speed for the 50km,50mi male to female

In [ ]:
df3.groupby(['race_length', 'athlete_gender'])['athlete_average_speed'].mean()

In [ ]:
# 2) What age groups are the best in the 50mi race (20 + race min) show 15

In [ ]:
df3.query('race_length == "50mi"').groupby('athlete_age')['athlete_average_speed'].agg(['mean', 'count']).sort_values('mean', ascending = False).query('count>19').head(15)

In [ ]:
# 3) What age groups are the worst in the 50mi Race (10 + races min) (show 20)

In [ ]:
df3.query('race_length == "50mi"').groupby('athlete_age')['athlete_average_speed'].agg(['mean', 'count']).sort_values('mean', ascending = True).query('count>10').head(20)

In [ ]:
# 4) Are runners slower in summer than winter?

In [ ]:
df3['race_month'] = df3['race_day'].str.split('.').str.get(1).astype(int)

In [ ]:
df3['race_season'] = df3['race_month'].apply(lambda x: 'Winter' if x > 11 else 'Fall' if x > 8 else 'Summer' if x > 5 else 'Spring' if x > 2 else 'Winter')

In [ ]:
df3.head(25)

In [ ]:
df3.groupby('race_season')['athlete_average_speed'].agg(['mean', 'count']).sort_values('mean', ascending = False)